# eBay car listing

Ad per this project I analyze EBay Kleinaznaigen dataset
(source: https://www.kaggle.com/orgesleka/used-cars-database/data)
Dataset contains information about German eBay listing in automotive category, cars in particular. Attributes of the data set look as provided per accompanied dictionary i.e.:

- dateCrawled : when this ad was first crawled, all field-values are taken from this date
- name : "name" of the car
- seller : private or dealer
offerType
- price : the price on the ad to sell the car
- abtest
- vehicleType
- yearOfRegistration : at which year the car was first registered
- gearbox
- powerPS : power of the car in PS
- model
- kilometer : how many kilometers the car has driven
- monthOfRegistration : at which month the car was first registered
- fuelType
- brand
- notRepairedDamage : if the car has a damage which is not repaired yet
- dateCreated : the date for which the ad at ebay was created
- nrOfPictures : number of pictures in the ad (unfortunately this field contains - everywhere a 0 and is thus useless (bug in crawler!) )
- postalCode
- lastSeenOnline : when the crawler saw this ad last online

## Goal of the project
Objective of the project is data discovery and general analysis in search for an interesting insights

In [49]:
# import libraries
import pandas as  pd
import numpy as np

In [50]:
#read csv
autos = pd.read_csv("autos.csv", encoding = "Latin-1")

In [51]:
autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


From general overview it is noticable the dataset has 50k entries in 20 columns, mostly stored as text. Putting aside with the description it is becomes clear that some of the columns could be stored as numbers, that would make the analysis possible e.g.:
- dateCrawled - should be datetime
- price - should be float
- odometer - should be int
- dateCreated - should be date
- lastSeen - should be datetime

Furthermore some of the column have null values that has to be fixed

## Data preparation
First step would be to change naming convention of columns into snakecase and also rename some of them
### Naming convention

In [52]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [53]:
naming_map = {
    "dateCrawled": "date_crawled",
    'offerType': "offer_type",
    "vehicleType" : "vehicle_type",
    "yearOfRegistration" : "registration_year",
    "powerPS" : "power_ps",
    "monthOfRegistration": "registration_month",
    "fuelType": "fuel_type",
    "notRepairedDamage": "unrepaired_damage",
    "dateCreated": "ad_created",
    "nrOfPictures": "nr_of_pictures",
    "postalCode": "postal_code",
    "lastSeen" : "last_seen"
}

autos.rename(naming_map, axis="columns", inplace=True)
autos.head(3)

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37


After using 'rename' method, right now column names match the preferred naming convention

### Converting numbers

In [54]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-04-04 16:40:33,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


Next thing I want to do is to convert numeric like columns into numeric type. Conversion are needed on following columns: price and odometer.

We can also notice that seller and offer_type columns almost entirely store the same value. Therefore we will remove those in further steps.


In [55]:
# remove "$" sign and "," separator from strings and convert into int
autos["price"] = autos["price"].str.replace("$","").str.replace(",","").astype(int)

In [56]:
autos["odometer"] = autos["odometer"].str.replace("km","").str.replace(",","").astype(int)
autos.rename({"odometer": "odometer_km"}, axis=1, inplace=True)

### Dropping columns

In [57]:
autos.drop(["seller", "offer_type"], axis=1, inplace=True)

### Translating data
Strings in datasets come from German website and therefore are coded in German. For the better clarity, I am gonna translate those values into English

In [58]:
col_to_translate = ["vehicle_type","gearbox","fuel_type","unrepaired_damage"]

dict_val = {}

for col in col_to_translate:
    val = autos.loc[autos[col].notnull(),col].unique()
    dict_val[col] = val

#autos.loc[:,["vehicle_type","gearbox","fuel_type","brand","unrepaired_damage"]]

In [59]:
dict_val

{'vehicle_type': array(['bus', 'limousine', 'kleinwagen', 'kombi', 'coupe', 'suv',
        'cabrio', 'andere'], dtype=object),
 'gearbox': array(['manuell', 'automatik'], dtype=object),
 'fuel_type': array(['lpg', 'benzin', 'diesel', 'cng', 'hybrid', 'elektro', 'andere'],
       dtype=object),
 'unrepaired_damage': array(['nein', 'ja'], dtype=object)}

In [60]:
# map as dict of dicts
translation_map = {
                    "gearbox" : {
                                "manuell" : "manual",
                                "automatik" : "automatic"
                                },
                    "unrepaired_damage" : {
                                "nein" : "no",
                                "ja" : "yes"
                            },
                    "vehicle_type" : {
                                'kleinwagen' : 'small car',
                                'andere' : 'other'
                    },
                    "fuel_type" : {
                                "benzin" : "gasoline",
                                "elektro" : "electric",
                                "andere" : "other"
                    }
}

#loop over each dict
for col in translation_map:
    #assign dict of dict per each column and keep defaults that are not in dict of dicts (fillna method)  
    autos[col] = autos[col].map(translation_map[col]).fillna(autos[col])
    

Now values in dataset are stored mostly in English

In [61]:
for col in col_to_translate:
    print(autos.loc[autos["gearbox"].notnull(),col].unique())


['bus' 'limousine' 'small car' 'kombi' nan 'coupe' 'suv' 'cabrio' 'other']
['manual' 'automatic']
['lpg' 'gasoline' 'diesel' nan 'cng' 'hybrid' 'electric' 'other']
['no' nan 'yes']


### Looking for disparity in data
#### prices

In [62]:
autos.describe()

,price,registration_year,power_ps,odometer_km,registration_month,nr_of_pictures,postal_code
count,5.000000e+04,50000.000000,50000.000000,50000.000000,50000.000000,50000.0,50000.000000
mean,9.840044e+03,2005.073280,116.355920,125732.700000,5.723360,0.0,50813.627300
std,4.811044e+05,105.712813,209.216627,40042.211706,3.711984,0.0,25779.747957
min,0.000000e+00,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1.100000e+03,1999.000000,70.000000,125000.000000,3.000000,0.0,30451.000000
50%,2.950000e+03,2003.000000,105.000000,150000.000000,6.000000,0.0,49577.000000
75%,7.200000e+03,2008.000000,150.000000,150000.000000,9.000000,0.0,71540.000000
max,1.000000e+08,9999.000000,17700.000000,150000.000000,12.000000,0.0,99998.000000


In [63]:
autos["price"].unique().shape

(2357,)

In [64]:
autos["price"].describe()

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64

Two middle quartiles of the prices are between 1100 and 7200 EUR. Figures in minimum and maximum though do no match (0 EUR and 100,000,000). Let's check case by case and decide what to do with them.

In [65]:
autos[autos["price"]<=1]

,date_crawled,name,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
27,2016-03-27 18:45:01,Hat_einer_Ahnung_mit_Ford_Galaxy_HILFE,0,control,NaN,2005,NaN,0,NaN,150000,0,NaN,ford,NaN,2016-03-27 00:00:00,0,66701,2016-03-27 18:45:01
55,2016-03-07 02:47:54,Mercedes_E320_AMG_zu_Tauschen!,1,test,NaN,2017,automatic,224,e_klasse,125000,7,gasoline,mercedes_benz,no,2016-03-06 00:00:00,0,22111,2016-03-08 05:45:44
71,2016-03-28 19:39:35,Suche_Opel_Astra_F__Corsa_oder_Kadett_E_mit_Re...,0,control,NaN,1990,manual,0,NaN,5000,0,gasoline,opel,NaN,2016-03-28 00:00:00,0,4552,2016-04-07 01:45:48
80,2016-03-09 15:57:57,Nissan_Primera_Hatchback_1_6_16v_73_Kw___99Ps_...,0,control,coupe,1999,manual,99,primera,150000,3,gasoline,nissan,yes,2016-03-09 00:00:00,0,66903,2016-03-09 16:43:50
87,2016-03-29 23:37:22,Bmw_520_e39_zum_ausschlachten,0,control,NaN,2000,NaN,0,5er,150000,0,NaN,bmw,NaN,2016-03-29 00:00:00,0,82256,2016-04-06 21:18:15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49884,2016-03-11 13:55:30,Audi_a6_2.5l__Schnaeppchen_nur_heute,0,test,kombi,1999,manual,150,a6,150000,11,diesel,audi,NaN,2016-03-11 00:00:00,0,27711,2016-03-12 03:17:08
49943,2016-03-16 20:46:08,Opel_astra,0,control,NaN,2016,manual,101,astra,150000,8,gasoline,opel,NaN,2016-03-16 00:00:00,0,89134,2016-03-17 19:44:20
49960,2016-03-25 22:51:55,Ford_KA_zu_verschenken_***Reserviert***,0,control,small car,1999,manual,60,ka,150000,6,gasoline,ford,NaN,2016-03-25 00:00:00,0,34355,2016-03-25 22:51:55
49974,2016-03-20 10:52:31,Golf_1_Cabrio_Tuev_Neu_viele_Extras_alles_eing...,0,control,cabrio,1983,manual,70,golf,150000,2,gasoline,volkswagen,no,2016-03-20 00:00:00,0,8209,2016-03-27 19:48:16


#### Lower range prices handling 
For the cars with the price not more than 1 EUR there are 1577 postions. 
Let's check the distribution of prices in ascending order.

In [66]:
autos["price"].value_counts().sort_index(ascending=True).head(20)

0     1421
1      156
2        3
3        1
5        2
8        1
9        1
10       7
11       2
12       3
13       2
14       1
15       2
17       3
18       1
20       4
25       5
29       1
30       7
35       1
Name: price, dtype: int64

Amount 1 EUR seems to be recurring. Trying to look for explanation we can check weather those cars are damaged (unrepaired_damage) that might suggest someone wants to just get rid of the car for no price. 


In [67]:
autos.loc[autos["price"]<1,"unrepaired_damage"].value_counts(dropna=False)

NaN    722
no     450
yes    249
Name: unrepaired_damage, dtype: int64

Unfortunately, damages explain only 249 cases while majority has no such an information stored in the data. We can only assume that those with no attribute covered are also vehicles that have either some sort of a problem or the incorrect price had been set up on purpose. Giving a context, those sort of operations are common on second hand trade walls. As a result ads are placed with the lowest possible price purposefully. People browsing website and sorting items by price ascending would see those before other value cars. 
It is therefore justified to just arbitrally remove rows with price not exceeding 1 EUR.


In [68]:
# removing postings lower or equal 1
autos = autos[autos["price"]>1]

#### Removing buy and exchange offers
Quick glance at the example data for a person slightly familiar with German language and Google Translate reveals one more issue. Plenty of adverts state in the title words like: "suchen", "tauchen", "kaufen" which means respectively "looking for", "exchange" and "will buy". There are also leasing cession offers. Those sort of posting are common on second hand trade walls and are placed with extreme low or high price.


In [69]:
# removing buy, exhange, leasing offers
to_remove_bool = autos['name'].str.lower().str.contains(r'(tausch.)|(suche.)|(kaufe.)|(leasing.)')

In [70]:
to_remove_bool.value_counts()

False    46311
True      2112
Name: name, dtype: int64

In [71]:
autos = autos[~to_remove_bool]

#### Upper range prices handling
On the other side of the problem we got superexpensive listings. Let's investigate those cases and remove the outliers.

In [72]:
autos["price"].value_counts().sort_index(ascending=False).head(20)

12345678    2
11111111    2
3890000     1
1300000     1
1234566     1
999999      2
999990      1
350000      1
345000      1
299000      1
295000      1
265000      1
259000      1
250000      1
220000      1
198000      1
197000      1
194000      1
190000      1
180000      1
Name: price, dtype: int64

Let's put a threshold at 999990 and take a look what happens there

In [73]:
autos.loc[autos["price"] >= 999990,["name","price"]]

,name,price
514,Ford_Focus_Turnier_1.6_16V_Style,999999
2897,Escort_MK_1_Hundeknochen_zum_umbauen_auf_RS_2000,11111111
7814,Ferrari_F40,1300000
22947,Bmw_530d_zum_ausschlachten,1234566
24384,Schlachte_Golf_3_gt_tdi,11111111
27371,Fiat_Punto,12345678
37585,Volkswagen_Jetta_GT,999990
43049,2_VW_Busse_T3,999999
47598,Opel_Vectra_B_1_6i_16V_Facelift_Tuning_Showcar...,12345678
47634,Ferrari_FXX,3890000


Similar story to underpriced adverts. Figures tend to have some sort of a pattern, either chain of a same digits or ascending chain. Seems like fraud again.
Nevertheless, there are 2 positions looking proper i.e. Ferrari F40 (row 7814) and Ferrari Fxx (row 47634). Those supercars in contrary to Ford Focus are likely to achieve such prices therefore we gonna exclude all adverts above 999990 except those two

In [74]:
# select all above 1M and not Ferrari
bool_discard = (autos["price"] >= 999990) & ~(autos["name"].str.contains("Ferrari"))
autos = autos[~bool_discard]
autos["price"].describe()

count    4.630300e+04
mean     6.148179e+03
std      2.112644e+04
min      2.000000e+00
25%      1.299000e+03
50%      3.200000e+03
75%      7.600000e+03
max      3.890000e+06
Name: price, dtype: float64

Prices distribution seems now to be reliably distributed. The outliers had been removed.

#### odometer

In [75]:
autos["odometer_km"].value_counts().sort_index(ascending=True)

5000        725
10000       242
20000       735
30000       757
40000       797
50000       994
60000      1133
70000      1187
80000      1385
90000      1679
100000     2034
125000     4886
150000    29749
Name: odometer_km, dtype: int64

Odometer values are given in ranges. We can conclude that from high end values of 5,000km at lower end and 150,000km at the higher end. We shall interpret the data as following:
    - 5000 means the vehicle has driven 5000km or less
    - 10000 means the vehicle has driven between 5001km and 10000km
    - (...)

#### date_crawled
Now I want to focus on datetime like columns. I will check the distribution and convert them into datetime format

In [76]:
date_crawled_string = autos["date_crawled"].str[:10]
date_crawled_string.value_counts(normalize=True, dropna=False).sort_index()

2016-03-05    0.025204
2016-03-06    0.014038
2016-03-07    0.036088
2016-03-08    0.032957
2016-03-09    0.033022
2016-03-10    0.032136
2016-03-11    0.032503
2016-03-12    0.036671
2016-03-13    0.015874
2016-03-14    0.036585
2016-03-15    0.034274
2016-03-16    0.029501
2016-03-17    0.031575
2016-03-18    0.013109
2016-03-19    0.034641
2016-03-20    0.037881
2016-03-21    0.037125
2016-03-22    0.033130
2016-03-23    0.032849
2016-03-24    0.029415
2016-03-25    0.031423
2016-03-26    0.032395
2016-03-27    0.031078
2016-03-28    0.034965
2016-03-29    0.034145
2016-03-30    0.033432
2016-03-31    0.031769
2016-04-01    0.034080
2016-04-02    0.035505
2016-04-03    0.038356
2016-04-04    0.036715
2016-04-05    0.013001
2016-04-06    0.003153
2016-04-07    0.001404
Name: date_crawled, dtype: float64

#### last_seen

In [77]:
last_seen_string = autos["last_seen"].str[:10]
last_seen_string.value_counts(normalize=True, dropna=False).sort_index()

2016-03-05    0.001080
2016-03-06    0.004341
2016-03-07    0.005399
2016-03-08    0.007192
2016-03-09    0.009632
2016-03-10    0.010410
2016-03-11    0.012353
2016-03-12    0.023541
2016-03-13    0.008898
2016-03-14    0.012591
2016-03-15    0.015658
2016-03-16    0.016392
2016-03-17    0.027817
2016-03-18    0.007429
2016-03-19    0.015766
2016-03-20    0.020560
2016-03-21    0.020582
2016-03-22    0.021165
2016-03-23    0.018724
2016-03-24    0.019372
2016-03-25    0.019005
2016-03-26    0.016824
2016-03-27    0.015571
2016-03-28    0.020776
2016-03-29    0.022158
2016-03-30    0.024664
2016-03-31    0.024081
2016-04-01    0.022806
2016-04-02    0.024556
2016-04-03    0.025160
2016-04-04    0.024404
2016-04-05    0.125499
2016-04-06    0.223247
2016-04-07    0.132346
Name: last_seen, dtype: float64

Crawler origined data is ranged between 5th March and 7th April 2016. Numbers for crawls are pretty much equally spreaded with around 3% share per day within scope.
'Last seen' attribute is not spreaded evenly however it is expected for the views to be higher in recent days than few weeks ago.

#### ad_created

In [78]:
ad_created_string = autos["ad_created"].str[:10]
ad_created_string.value_counts(normalize=True, dropna=False).sort_index()

ad_created_string.value_counts(dropna=False).sort_index().head(30)

2015-06-11    1
2015-08-10    1
2015-11-10    1
2015-12-05    1
2015-12-30    1
2016-01-03    1
2016-01-07    1
2016-01-10    2
2016-01-13    1
2016-01-14    1
2016-01-16    1
2016-01-22    1
2016-01-27    2
2016-01-29    1
2016-02-01    1
2016-02-02    2
2016-02-05    2
2016-02-07    1
2016-02-08    1
2016-02-09    1
2016-02-11    1
2016-02-12    2
2016-02-14    2
2016-02-16    1
2016-02-17    1
2016-02-18    2
2016-02-19    2
2016-02-20    2
2016-02-21    3
2016-02-22    1
Name: ad_created, dtype: int64

created_at date is wesbsite own data and corresponds to date of ad creation. Numbers resembles those stored in date_crawled however we have significant shares of those that were created before we started crawling. Some entries come from beginning of 2016 and there are 6 cases started in 2nd half of 2015. As much as it might seem off, when checking eBay indeed there are offers that are 1 or 2 years old. There is no reason then to get rid of those entries.

#### registration year

In [79]:
autos['registration_year'].describe()

count    46303.000000
mean      2004.824612
std         89.688248
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

Registration year stats reveal problems in min and max values. It is necessary to inspect records below and above certain dates.

In [80]:
autos[~autos["registration_year"].between(1906,2016)]

,date_crawled,name,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
10,2016-03-15 01:41:36,VW_Golf_Tuning_in_siber/grau,999,test,NaN,2017,manual,90,NaN,150000,4,gasoline,volkswagen,no,2016-03-14 00:00:00,0,86157,2016-04-07 03:16:21
65,2016-04-04 19:30:39,Ford_Fiesta_zum_ausschlachten,250,control,NaN,2017,manual,65,fiesta,125000,9,gasoline,ford,NaN,2016-04-04 00:00:00,0,65606,2016-04-05 12:22:12
68,2016-04-03 17:36:59,Mini_cooper_s_clubman_/vollausstattung_/_Navi/...,10990,test,NaN,2017,manual,174,clubman,100000,0,NaN,mini,no,2016-04-03 00:00:00,0,83135,2016-04-05 17:26:26
84,2016-03-27 19:52:54,Renault_twingo,900,control,NaN,2018,NaN,60,twingo,150000,0,NaN,renault,NaN,2016-03-27 00:00:00,0,40589,2016-04-05 18:46:49
113,2016-04-03 14:58:29,Golf_4_Anfaenger_auto,1200,test,NaN,2017,manual,75,golf,150000,7,NaN,volkswagen,NaN,2016-04-03 00:00:00,0,97656,2016-04-05 14:15:48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49731,2016-03-28 13:40:31,Renault_MEGANE_1.6_16V__neu_ZANHRIEMEN_TOP_ZUS...,1700,test,NaN,2017,manual,0,megane,150000,7,NaN,renault,no,2016-03-28 00:00:00,0,34123,2016-03-31 21:44:52
49770,2016-03-15 12:54:26,VW_Polo_6n_Tuev_Neu!__1.6_75PS,999,control,NaN,2018,manual,75,polo,150000,12,gasoline,volkswagen,no,2016-03-15 00:00:00,0,24321,2016-04-06 02:16:02
49841,2016-03-11 15:37:02,Passat_abzugeben.,600,test,NaN,2017,manual,101,passat,150000,7,NaN,volkswagen,NaN,2016-03-11 00:00:00,0,53804,2016-03-11 16:41:14
49910,2016-04-03 21:39:15,Schoener_fast_neuer_Opel_Mokka_in_Zell_Mosel_m...,22200,test,NaN,9000,automatic,140,andere,10000,3,gasoline,opel,NaN,2016-04-03 00:00:00,0,56856,2016-04-05 22:18:26


1878 rows with registration year out of scope [1906,2016]. Listings in our data (from Jan-Feb 2016) logically should not have registration date after Feb 2016. 

Same for cars registered before 1906 (the year of first regitration plates in Germany). 

We shall remove those data points from our set.

In [81]:
autos = autos[autos["registration_year"].between(1906,2016)]


## Brand statistics

In [82]:
autos['brand'].value_counts(normalize=True).sort_values(ascending=False)#.head(5).sum()

volkswagen        0.209951
bmw               0.110618
opel              0.106580
mercedes_benz     0.098077
audi              0.087332
ford              0.069184
renault           0.046526
peugeot           0.030330
fiat              0.025820
seat              0.018305
skoda             0.016556
nissan            0.015165
mazda             0.015030
smart             0.014312
citroen           0.014110
toyota            0.012989
hyundai           0.010162
sonstige_autos    0.009085
mini              0.009018
volvo             0.008973
mitsubishi        0.008368
honda             0.007829
kia               0.007066
alfa_romeo        0.006506
porsche           0.006259
suzuki            0.005877
chevrolet         0.005810
chrysler          0.003432
dacia             0.002670
daihatsu          0.002513
jeep              0.002311
land_rover        0.002154
subaru            0.002086
saab              0.001682
jaguar            0.001615
daewoo            0.001436
trabant           0.001324
r

#### Brand popularity
From the stats we can see the the vast of the car listings are the truly German cars (Volkswagen, BMW, Audi, Opel, Mercedes) standing for 61% of all adverts. In the latter part I want to analyze those brands pricewise and determine the stats for price and year of registration for each brand.

In [83]:
brands = ["volkswagen","bmw","opel","mercedes_benz","audi"]
brands_av_price = [] 
brands_av_year = []

for brand in brands:
     average_price = autos.loc[autos["brand"]==brand,"price"].describe()
     average_year = autos.loc[autos["brand"]==brand,"registration_year"].describe()
     brands_av_price.append(average_price)
     brands_av_year.append(average_year)
     
brands_average_price = pd.DataFrame(brands_av_price, index = brands)
brands_average_year = pd.DataFrame(brands_av_year, index = brands)


#### Brand mean price

In [84]:
brands_average_price

,count,mean,std,min,25%,50%,75%,max
volkswagen,9359.0,5562.403569,6304.339460,5.0,1300.0,3150.0,7799.5,64500.0
bmw,4931.0,8492.078483,9962.713006,12.0,2465.0,5999.0,11500.0,259000.0
opel,4751.0,3047.360977,3653.113315,3.0,800.0,1699.0,3900.0,38990.0
mercedes_benz,4372.0,8728.560842,10256.112991,17.0,2390.0,5200.0,11900.0,180000.0
audi,3893.0,9500.490367,10104.459340,11.0,2400.0,6500.0,13000.0,175000.0


In [85]:
average_prices = brands_average_price["mean"]
av_mileage = {}

for brand in brands:
    av_odo = autos.loc[autos["brand"]==brand,"odometer_km"].mean()
    av_mileage[brand] = av_odo

average_mileage = pd.Series(av_mileage)
average_mileage

average_compare = pd.DataFrame([average_prices,average_mileage], index=["price","mileage"])
average_compare

,volkswagen,bmw,opel,mercedes_benz,audi
price,5562.403569,8492.078483,3047.360977,8728.560842,9500.490367
mileage,128446.415215,132605.962279,129163.334035,130784.537969,128892.884665


From the summary table we can analyze the differences in prices among 5 German brands. As we see from the second table the price does not really correlate with the price as all mileage values are about range 129 - 132 kkm

- Volkswagen the most numerous brand on average costs 5418 EUR and 2 middle quartiles are within 1200 and 7500 EUR. Themost expensive car is priced at less then 65k
- BMW on average is more expensive by almost 3k. Half of the cars are ranged between 2390 and 11000
- Opel is definately the cheapest in the comparison ranging mostly between 1650 and 3800 with average of 3610 EUR
- Mercedes has very similar numbers to BMW having slightly higher average (8657). The prices of this brand also has the largest standard deviations which can be linked to the faact that even though a premium brand there are still a lot of old cars that are priced at the low range
- Audi while having the highest average it also got a highest middle quartile reaching 6400 EUR. Lowest quartile is 2300, similarly to Mercedes and BMW, exceeding in Q3 reaching almost 13k (highest amount)

The most expensive car among that group belongs to BMW at it is BMW Z8 Roadster from 2001 stating 400HP and not more than 20kkm driven.

In [86]:
autos[autos["price"]==259000]

,date_crawled,name,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
47337,2016-04-05 10:25:38,BMW_Z8_roadster,259000,test,cabrio,2001,manual,400,z_reihe,20000,6,gasoline,bmw,no,2016-04-05 00:00:00,0,61462,2016-04-05 12:07:32


#### Brand average car age

In [87]:
brands_average_year

,count,mean,std,min,25%,50%,75%,max
volkswagen,9359.0,2002.611604,7.214697,1943.0,1998.0,2003.0,2008.0,2016.0
bmw,4931.0,2003.075441,6.297621,1929.0,1999.0,2004.0,2008.0,2016.0
opel,4751.0,2002.261208,6.750888,1910.0,1998.0,2002.0,2006.0,2016.0
mercedes_benz,4372.0,2002.098582,7.759401,1937.0,1999.0,2003.0,2007.0,2016.0
audi,3893.0,2004.261238,6.198069,1968.0,2000.0,2005.0,2009.0,2016.0


Comparing years of registration the data values are similar among brands. Majority of vehicles were registered between 1998 and 2009. Opel tends to be the oldest one (2nd quartile 2002) and Audi has the newst cars listings (2nd quartile 2005)

### Most common brand/model combination

In [88]:
autos["brand/model"] = autos["brand"] + " " + autos["model"]

top_models = autos["brand/model"].value_counts().sort_values(ascending=False)[:20]
top_models = pd.DataFrame(top_models,index=top_models.index)
top_models.head()

,brand/model
volkswagen golf,3513
bmw 3er,2521
volkswagen polo,1510
opel corsa,1487
opel astra,1296


## Price correlation with mileage

I want to check weather mileage of the car has a negative correlation with the price. For that I use top four brand/model combination

In [89]:
top_models = autos["brand/model"].value_counts().sort_values(ascending=False)[:4]
filter = top_models.index
models_bool = autos["brand/model"].isin(filter)

In [90]:
price_mileage = autos.loc[models_bool,["brand/model","odometer_km","price"]].groupby(["brand/model","odometer_km"])["price"].mean()
pd.DataFrame(price_mileage)

price
brand/model     odometer_km              
bmw 3er         5000          6701.787879
                10000        23055.400000
                20000        11533.437500
                30000        23192.375000
                40000        29165.500000
                50000        22695.736842
                60000        17194.884615
                70000        17597.888889
                80000        15111.051282
                90000        13435.676923
                100000       11140.215190
                125000        8807.534759
                150000        4422.451904
opel corsa      5000          1655.960000
                10000         8090.000000
                20000         6779.235294
                30000         7279.083333
                40000         5896.000000
                50000         6454.275862
                60000         5163.875000
                70000         4746.758621
                80000         3990.156250
                90000         2924.964286
                100000        2448.615385
                125000        2319.831776
                150000        1088.614165
volkswagen golf 5000          3699.250000
                10000        21439.421053
                20000        19263.015873
                30000        17047.031250
                40000        15935.385965
                50000        14400.910448
                60000        13114.146341
                70000        12233.080000
                80000        10244.427083
                90000         8697.857143
                100000        7306.107692
                125000        5900.057402
                150000        2878.442671
volkswagen polo 5000          2814.916667
                10000        10207.000000
                20000         9263.470588
                30000         8285.954545
                40000         9502.176471
                50000         7845.666667
                60000         6428.777778
                70000         5813.605263
                80000         5481.650000
                90000         5215.474576
                100000        3888.984615
                125000        3152.590164
                150000        1383.451913

Generally speaking the negative correlation between mileage and price exists. What we excpect is drop of the price when the mileage is larger. 

We can spot however that in some cases we have unexplained behaviors. That behavior repeats for cars with the lowest mileage (5000). Other issues can be noticed with odometer at between 20 and 30k for some cars. We can expect that cars that have low value and low mileage can be damaged. That shall be checked at the next step.

In [91]:
unrepaired_bool = autos["unrepaired_damage"] == "yes"
unrepaired_prices = autos.loc[models_bool & unrepaired_bool,["brand/model","odometer_km","unrepaired_damage","price"]].groupby(["brand/model","odometer_km","unrepaired_damage"])["price"].mean()
repaired_prices = autos.loc[models_bool & ~unrepaired_bool,["brand/model","odometer_km","unrepaired_damage","price"]].groupby(["brand/model","odometer_km","unrepaired_damage"])["price"].mean()

unrepaired_prices

brand/model      odometer_km  unrepaired_damage
bmw 3er          5000         yes                   1283.333333
                 30000        yes                   3850.000000
                 60000        yes                   2224.500000
                 70000        yes                   4500.000000
                 90000        yes                   4166.666667
                 100000       yes                   8749.500000
                 125000       yes                   1598.000000
                 150000       yes                   2321.586957
opel corsa       5000         yes                    350.000000
                 20000        yes                   2237.500000
                 30000        yes                   8100.000000
                 50000        yes                   6924.750000
                 60000        yes                   3774.500000
                 70000        yes                   3850.000000
                 80000        yes                   3333

In [92]:
repaired_prices

brand/model      odometer_km  unrepaired_damage
bmw 3er          5000         no                   10328.769231
                 10000        no                   23055.400000
                 20000        no                   15426.000000
                 30000        no                   30657.800000
                 40000        no                   29165.500000
                 50000        no                   22695.736842
                 60000        no                   18442.416667
                 70000        no                   18524.969697
                 80000        no                   15299.513514
                 90000        no                   14773.157895
                 100000       no                   11394.070423
                 125000       no                    9213.529762
                 150000       no                    5001.533647
opel corsa       5000         no                    3567.666667
                 10000        no                    8260

In case of the upper table of damaged cars clearly there is no pattern. Understandable as in such cases it's more the state (or what's left) and equipment of the car that decides the price.

For non damaged cars, even though the table looks slightly better, there are still inconsistencies
- <5kkm are still the cheapest cars
- 10kkm - 20kkm do sometimes do not diminish respectively

The solution is the context knowledge and website research. Going on ebay website and looking for "real occasions" shows that the super expensive and brand new cars are sometimes posted with very low price to draw attention, but in the description only we can note that it is just the first installment or one month lease.

## Summary
Performed analysis of the second hand cars from German eBay bring interesting insights. 

- To start with we clearly see that Germans really love their cars as the Top5 German brands states for over 60% of all adverts.
- We now know that Opel is the cheapest car having average price not exceeding 3,000 EUR and being chased by the car for people "Volkswagen" that is less then 6,000 EUR on second hand listings. Most expensive is Audi being over 9,000 EUR on average.
- We proved that as logic suggests the higher the mileage the lower the price is.
- We know that outstanding leader in number of offers on eBay is Volkswagen Golf being chased by BMW 3.
- It was also an interesting discovery to reveal the scale of frauds and adverts that purposefully have the price in the listing lower than real value of the car.